# Microservice Catalog

Zeigt die Produktdaten an.

Diese Notebook erstellt aus dem Notebook `catalog` ein Container Image.

In [ ]:
import os
MICROSERVICE="catalog"
os.environ['MICROSERVICE']=MICROSERVICE

In [ ]:
%run ../export-notebook.py $MICROSERVICE".ipynb" $MICROSERVICE".py"

In [ ]:
! cat $MICROSERVICE.py

Zusätzlich brauchen wir ein `Dockerfile`. 

Das erstellen wir anhand eines Templates und ersetzen nur den Namen des Python Scripts.


In [ ]:
%run ../generate_dockerfile.py $MICROSERVICE

In [ ]:
! cat Dockerfile

In [ ]:
! docker build -t $MICROSERVICE .

In [ ]:
! docker run -d --rm --name $MICROSERVICE -p8080:8080 $MICROSERVICE

Export des Images ins TAR-Format um des anschliessend wieder in microk8s importieren zu können

In [ ]:
! docker save $MICROSERVICE -o $MICROSERVICE.tar
! microk8s ctr image import $MICROSERVICE.tar

In [ ]:
! microk8s ctr image list | awk '{print $1, $4, $5, $6}' | grep $MICROSERVICE

Und starten des Images in Kubernetes

In [ ]:
! kubectl run catalog --image $MICROSERVICE --image-pull-policy="IfNotPresent" --restart=Never 
! kubectl expose pod/$MICROSERVICE --type="LoadBalancer" --port 8080 

In [ ]:
! echo "http://"$(cat ~/work/server-ip)":$(kubectl get service $MICROSERVICE -o=jsonpath='{ .spec.ports[0].nodePort }')/"

In [ ]:
! docker stop $MICROSERVICE
! kubectl delete pod/$MICROSERVICE
! kubectl delete service/$MICROSERVICE
! rm -rf $MICROSERVICE.py Dockerfile $MICROSERVICE.tar